In [54]:
import sys
sys.path.append('/home/debo/learn2learn/')
sys.path.append('../../learn2learn/')
from torch.autograd import grad
from utils import clone_module
from torch import nn
import random
import numpy as np

import torch
from torch import nn
from torch import optim

import learn2learn as l2l

from scipy.stats import truncnorm
from torch.optim import Adam

In [38]:
ways=5
shots=1
meta_lr=0.003
fast_lr=0.5
meta_batch_size=32
adaptation_steps=1
num_iterations=60000
cuda=True
seed=42

In [39]:

from PIL.Image import LANCZOS
from torchvision.datasets import Omniglot
from torchvision import transforms
from torch.utils.data import ConcatDataset

random.seed(seed)
np.random.seed(seed)
th.manual_seed(seed)
device = th.device('cpu')
if cuda:
    th.cuda.manual_seed(seed)
    device = th.device('cuda')

# Create Dataset
# TODO: Create l2l.data.vision.FullOmniglot, which merges background and evaluation sets.
omni_background = Omniglot(root='./data',
                           background=True,
                           transform=transforms.Compose([
                               transforms.Resize(28, interpolation=LANCZOS),
                               transforms.ToTensor(),
                               # TODO: Add DiscreteRotations([0, 90, 180, 270])
                               lambda x: 1.0 - x,
                           ]),
                           download=True)
#    max_y = 1 + max([y for X, y in omni_background])
max_y = 964
omni_evaluation = Omniglot(root='./data',
                           background=False,
                           transform=transforms.Compose([
                               transforms.Resize(28, interpolation=LANCZOS),
                               transforms.ToTensor(),
                               # TODO: Add DiscreteRotations([0, 90, 180, 270])
                               lambda x: 1.0 - x,
                           ]),
                           target_transform=transforms.Compose([
                               lambda x: max_y + x,
                           ]),
                           download=True)
omniglot = ConcatDataset((omni_background, omni_evaluation))
train_generator = l2l.data.TaskGenerator(dataset=omniglot, ways=ways)
valid_generator = l2l.data.TaskGenerator(dataset=omniglot, ways=ways)
test_generator = l2l.data.TaskGenerator(dataset=omniglot, ways=ways)
# TODO: Implement an easy way to split one dataset into splits, based on classes.


Files already downloaded and verified
Files already downloaded and verified


In [41]:
num_input_channels = 1
def conv_block(in_channels: int, out_channels: int) -> nn.Module:
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
model = nn.Sequential(
        conv_block(num_input_channels, 64),
        conv_block(64, 64),
        conv_block(64, 64),
        conv_block(64, 64),
        Flatten(),
    )
model.to(device, dtype=torch.double)

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05

In [119]:
import torch
from torch.optim import Optimizer
from torch.nn import Module
from typing import Callable
import torch


def pairwise_distances(x: torch.Tensor,
                       y: torch.Tensor,
                       matching_fn: str) -> torch.Tensor:
    """Efficiently calculate pairwise distances (or other similarity scores) between
    two sets of samples.

    # Arguments
        x: Query samples. A tensor of shape (n_x, d) where d is the embedding dimension
        y: Class prototypes. A tensor of shape (n_y, d) where d is the embedding dimension
        matching_fn: Distance metric/similarity score to compute between samples
    """
    n_x = x.shape[0]
    n_y = y.shape[0]

    if matching_fn == 'l2':
        distances = (
                x.unsqueeze(1).expand(n_x, n_y, -1) -
                y.unsqueeze(0).expand(n_x, n_y, -1)
        ).pow(2).sum(dim=2)
        return distances
    elif matching_fn == 'cosine':
        normalised_x = x / (x.pow(2).sum(dim=1, keepdim=True).sqrt() + EPSILON)
        normalised_y = y / (y.pow(2).sum(dim=1, keepdim=True).sqrt() + EPSILON)

        expanded_x = normalised_x.unsqueeze(1).expand(n_x, n_y, -1)
        expanded_y = normalised_y.unsqueeze(0).expand(n_x, n_y, -1)

        cosine_similarities = (expanded_x * expanded_y).sum(dim=2)
        return 1 - cosine_similarities
    elif matching_fn == 'dot':
        expanded_x = x.unsqueeze(1).expand(n_x, n_y, -1)
        expanded_y = y.unsqueeze(0).expand(n_x, n_y, -1)

        return -(expanded_x * expanded_y).sum(dim=2)
    else:
        raise(ValueError('Unsupported similarity function'))




def proto_net_episode(model: Module,
                      optimiser: Optimizer,
                      loss_fn: Callable,
                      x: torch.Tensor,
                      y: torch.Tensor,
                      n_shot: int,
                      k_way: int,
                      q_queries: int,
                      distance: str,
                      train: bool):
    """Performs a single training episode for a Prototypical Network.

    # Arguments
        model: Prototypical Network to be trained.
        optimiser: Optimiser to calculate gradient step
        loss_fn: Loss function to calculate between predictions and outputs. Should be cross-entropy
        x: Input samples of few shot classification task
        y: Input labels of few shot classification task
        n_shot: Number of examples per class in the support set
        k_way: Number of classes in the few shot classification task
        q_queries: Number of examples per class in the query set
        distance: Distance metric to use when calculating distance between class prototypes and queries
        train: Whether (True) or not (False) to perform a parameter update

    # Returns
        loss: Loss of the Prototypical Network on this task
        y_pred: Predicted class probabilities for the query set on this task
    """
    if train:
        # Zero gradients
        model.train()
        optimiser.zero_grad()
    else:
        model.eval()

    # Embed all samples
    embeddings = model(x)

    # Samples are ordered by the NShotWrapper class as follows:
    # k lots of n support samples from a particular class
    # k lots of q query samples from those classes
    support = embeddings[:n_shot*k_way]
    queries = embeddings[n_shot*k_way:]
    prototypes = compute_prototypes(support, k_way, n_shot)

    # Calculate squared distances between all queries and all prototypes
    # Output should have shape (q_queries * k_way, k_way) = (num_queries, k_way)
    distances = pairwise_distances(queries, prototypes, distance)

    # Calculate log p_{phi} (y = k | x)
    log_p_y = (-distances).log_softmax(dim=1)

    print('size is ', log_p_y)
    print('size y is ', y)
    loss = loss_fn(log_p_y, y)

    # Prediction probabilities are softmax over distances
    y_pred = (-distances).softmax(dim=1)

    if train:
        # Take gradient step
        loss.backward()
        optimiser.step()
    else:
        pass

    return loss, y_pred


def compute_prototypes(support: torch.Tensor, k: int, n: int) -> torch.Tensor:
    """Compute class prototypes from support samples.

    # Arguments
        support: torch.Tensor. Tensor of shape (n * k, d) where d is the embedding
            dimension.
        k: int. "k-way" i.e. number of classes in the classification task
        n: int. "n-shot" of the classification task

    # Returns
        class_prototypes: Prototypes aka mean embeddings for each class
    """
    # Reshape so the first dimension indexes by class then take the mean
    # along that dimension to generate the "prototypes" for each class
    class_prototypes = support.reshape(k, n, -1).mean(dim=1)
    return class_prototypes

In [100]:
from PIL.Image import LANCZOS
from torchvision.datasets import Omniglot
from torchvision import transforms
from torch.utils.data import ConcatDataset

random.seed(seed)
np.random.seed(seed)
th.manual_seed(seed)
device = th.device('cpu')
if cuda:
    th.cuda.manual_seed(seed)
    device = th.device('cuda')

# Create Dataset
# TODO: Create l2l.data.vision.FullOmniglot, which merges background and evaluation sets.
omni_background = Omniglot(root='./data',
                           background=True,
                           transform=transforms.Compose([
                               transforms.Resize(28, interpolation=LANCZOS),
                               transforms.ToTensor(),
                               # TODO: Add DiscreteRotations([0, 90, 180, 270])
                               lambda x: 1.0 - x,
                           ]),
                           download=True)
#    max_y = 1 + max([y for X, y in omni_background])
max_y = 964
omni_evaluation = Omniglot(root='./data',
                           background=False,
                           transform=transforms.Compose([
                               transforms.Resize(28, interpolation=LANCZOS),
                               transforms.ToTensor(),
                               # TODO: Add DiscreteRotations([0, 90, 180, 270])
                               lambda x: 1.0 - x,
                           ]),
                           target_transform=transforms.Compose([
                               lambda x: max_y + x,
                           ]),
                           download=True)
omniglot = ConcatDataset((omni_background, omni_evaluation))
train_generator = l2l.data.TaskGenerator(dataset=omniglot, ways=ways)
valid_generator = l2l.data.TaskGenerator(dataset=omniglot, ways=ways)
test_generator = l2l.data.TaskGenerator(dataset=omniglot, ways=ways)

Files already downloaded and verified
Files already downloaded and verified


In [65]:
adaptation_data = train_generator.sample(shots=shots)

In [91]:
q1 = torch.stack(adaptation_data.data).squeeze()

In [92]:
q1.shape

torch.Size([5, 28, 28])

In [90]:
q2.shape

torch.Size([5, 28, 28])

In [84]:
adaptation_data.data[3].shape

torch.Size([1, 28, 28])

In [69]:
adaptation_data.label

[3, 0, 4, 2, 1]

In [134]:
adaptation_data = train_generator.sample(shots=shots)
evaluation_data = train_generator.sample(shots=shots, classes_to_sample=adaptation_data.sampled_classes)
support_x = torch.stack(adaptation_data.data).double().to(device)
evaluation_x = torch.stack(evaluation_data.data).double().to(device)
y = torch.DoubleTensor(adaptation_data.label)
n_shot = shots
k_way = ways
distance='l2'
train=True
if train:
    # Zero gradients
    model.train()
    optimiser.zero_grad()
else:
    model.eval()

# Embed all samples
embeddings = model(x)

# Samples are ordered by the NShotWrapper class as follows:
# k lots of n support samples from a particular class
# k lots of q query samples from those classes
support = model(support_x)
queries = model(evaluation_x)
prototypes = compute_prototypes(support, k_way, n_shot)

In [143]:
# Calculate squared distances between all queries and all prototypes
# Output should have shape (q_queries * k_way, k_way) = (num_queries, k_way)
distances = pairwise_distances(queries, prototypes, distance)

# Calculate log p_{phi} (y = k | x)
log_p_y = (-distances).log_softmax(dim=1).to(device)
y = y.long()

print('size is ', log_p_y)
print('size y is ', y)
loss = loss_fn(log_p_y, y)

# Prediction probabilities are softmax over distances
y_pred = (-distances).softmax(dim=1)

if train:
    # Take gradient step
    loss.backward()
    optimiser.step()
else:
    pass

print('this should work', loss, y_pred)

optimiser = Adam(model.parameters(), lr=1e-3)
loss_fn = torch.nn.NLLLoss().cuda()
batch_support = torch.stack(adaptation_data.data).double().to(device)
batch_label = torch.DoubleTensor(adaptation_data.label)
proto_net_episode(model=model,optimiser=optimiser,loss_fn=loss_fn,x=batch_support,
                  y=batch_label,
                n_shot=shots,
                      k_way=ways,
                      q_queries=shots,
                      distance='l2',
                      train=True)

size is  tensor([[-1.2249e-03, -1.4098e+01, -3.9641e+01, -1.2086e+01, -6.7107e+00],
        [-2.7582e+00, -7.9253e-02, -4.3593e+00, -1.5438e+01, -1.9832e+01],
        [-2.2861e+01, -1.3791e+01, -1.0247e-06, -3.4640e+01, -5.5083e+01],
        [-4.7291e+00, -8.1878e+00, -1.6292e+01, -9.1559e-03, -1.3419e+01],
        [-3.1144e-01, -9.3519e+00, -2.9496e+01, -7.3150e+00, -1.3210e+00]],
       device='cuda:0', dtype=torch.float64, grad_fn=<LogSoftmaxBackward>)
size y is  tensor([4, 1, 3, 2, 0], device='cuda:0')
this should work tensor(11.6066, device='cuda:0', dtype=torch.float64,
       grad_fn=<NllLossBackward>) tensor([[9.9878e-01, 7.5359e-07, 6.0819e-18, 5.6387e-06, 1.2178e-03],
        [6.3406e-02, 9.2381e-01, 1.2788e-02, 1.9733e-07, 2.4375e-09],
        [1.1791e-10, 1.0246e-06, 1.0000e+00, 9.0403e-16, 1.1955e-24],
        [8.8345e-03, 2.7802e-04, 8.4033e-08, 9.9089e-01, 1.4866e-06],
        [7.3239e-01, 8.6798e-05, 1.5491e-13, 6.6548e-04, 2.6686e-01]],
       device='cuda:0', dtype=to

ValueError: Expected input batch_size (0) to match target batch_size (5).

In [111]:
p = torch.Tensor([[1.,2., 3.]])

In [114]:
p.size()

torch.Size([1, 3])

In [110]:
p.reshape(3)

tensor([1., 2., 3.])

In [36]:
def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)

In [28]:
def fast_adapt(adaptation_data, evaluation_data, learner, loss, adaptation_steps, device):
    for step in range(adaptation_steps):
        data = [d for d in adaptation_data]
        X = th.cat([d[0] for d in data], dim=0).to(device)
        y = th.cat([th.tensor(d[1]).view(-1) for d in data], dim=0).to(device)
        train_error = loss(learner(X), y)
        train_error /= len(adaptation_data)
        learner.adapt(train_error)
    data = [d for d in evaluation_data]
    X = th.cat([d[0] for d in data], dim=0).to(device)
    y = th.cat([th.tensor(d[1]).view(-1) for d in data], dim=0).to(device)
    predictions = learner(X)
    valid_error = loss(predictions, y)
    valid_error /= len(evaluation_data)
    valid_accuracy = accuracy(predictions, y)
    return valid_error, valid_accuracy

In [30]:
model.to(device, dtype=torch.double)
ptnet = ProtoNet(model, lr=fast_lr, first_order=False)
opt = optim.Adam(model.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(size_average=True, reduction='mean')

for iteration in range(num_iterations):
    opt.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0
    meta_test_error = 0.0
    meta_test_accuracy = 0.0
    for task in range(meta_batch_size):
        # Compute meta-training loss
        learner = ptnet.clone()
        adaptation_data = train_generator.sample(shots=shots)
        evaluation_data = train_generator.sample(shots=shots,
                                                 classes_to_sample=adaptation_data.sampled_classes)
        evaluation_error, evaluation_accuracy = fast_adapt(adaptation_data,
                                                           evaluation_data,
                                                           learner,
                                                           loss,
                                                           adaptation_steps,
                                                           device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()

        # Compute meta-validation loss
        learner = ptnet.clone()
        adaptation_data = valid_generator.sample(shots=shots)
        evaluation_data = valid_generator.sample(shots=shots,
                                                 classes_to_sample=adaptation_data.sampled_classes)
        evaluation_error, evaluation_accuracy = fast_adapt(adaptation_data,
                                                           evaluation_data,
                                                           learner,
                                                           loss,
                                                           adaptation_steps,
                                                           device)
        meta_valid_error += evaluation_error.item()
        meta_valid_accuracy += evaluation_accuracy.item()

        # Compute meta-testing loss
        learner = ptnet.clone()
        adaptation_data = test_generator.sample(shots=shots)
        evaluation_data = test_generator.sample(shots=shots,
                                                classes_to_sample=adaptation_data.sampled_classes)
        evaluation_error, evaluation_accuracy = fast_adapt(adaptation_data,
                                                           evaluation_data,
                                                           learner,
                                                           loss,
                                                           adaptation_steps,
                                                           device)
        meta_test_error += evaluation_error.item()
        meta_test_accuracy += evaluation_accuracy.item()

    # Print some metrics
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
    print('Meta Train Accuracy', meta_train_accuracy / meta_batch_size)
    print('Meta Valid Error', meta_valid_error / meta_batch_size)
    print('Meta Valid Accuracy', meta_valid_accuracy / meta_batch_size)
    print('Meta Test Error', meta_test_error / meta_batch_size)
    print('Meta Test Accuracy', meta_test_accuracy / meta_batch_size)

    # Average the accumulated gradients and optimize
    for p in ptnet.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()



/data/anaconda/envs/fewshot/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


RuntimeError: Expected 4-dimensional input for 4-dimensional weight 64 1 3, but got 3-dimensional input of size [5, 28, 28] instead